# This the third step in the medallion architecture - the Gold layer - Creating a schema that is suitable for reporting(Star schema) with Date, Customer and Product as dimensions  and Sales as a fact table


In [3]:
# Step 1 read the silver table
df = spark.read.table("dbo.sales_silver")
display(df.limit(10))

StatementMeta(, 5513f65f-7feb-4b25-aaa2-0cc93b52509b, 5, Finished, Available, Finished)

SynapseWidget(Synapse.DataFrame, c1019290-c5d9-4fbc-b0ab-df422e5a8d33)

# create a date dimension table

In [13]:
from pyspark.sql.types import *
from delta.tables import*

DeltaTable.createIfNotExists(spark)\
.tableName("dbo.dimdate_gold") \
.addColumn("OrderDate", DateType()) \
.addColumn("Day", IntegerType()) \
.addColumn("Month", IntegerType()) \
.addColumn("Year", IntegerType()) \
.addColumn("mmmyyyy", StringType())\
.addColumn("yyyymm", StringType())\
.execute()

StatementMeta(, f1faf6aa-3ec2-49b0-8dc9-20c88f5228d1, 15, Finished, Available, Finished)

In [14]:
from pyspark.sql.functions import * 
# creating data frame for date 

dfdimDate_gold =  df.dropDuplicates(["OrderDate"]).select(col("OrderDate"),\
dayofmonth("OrderDate").alias("Day"), \
month("OrderDate").alias("Month"), \
year("OrderDate").alias("Year"), \
date_format(col("OrderDate"),"MMM-yyyy").alias("mmmyyyy"), \
date_format(col("OrderDate"), "yyyyMM").alias("yyyymm"), \
).orderBy("OrderDate")

display(dfdimDate_gold.limit(10))

StatementMeta(, f1faf6aa-3ec2-49b0-8dc9-20c88f5228d1, 16, Finished, Available, Finished)

SynapseWidget(Synapse.DataFrame, 054c54b4-3fe7-42d3-8bb3-6376310273e5)

In [18]:
from delta.tables import *
deltaTable = DeltaTable.forPath(spark, 'Tables/dbo/dimdate_gold')
dfUpdates = dfdimDate_gold

deltaTable.alias('gold') \
.merge(
    dfUpdates.alias('updates'),
    'gold.OrderDate = updates.OrderDate'
)\
.whenMatchedUpdate(set=
{

}
)\
.whenNotMatchedInsert(values =
{
    "OrderDate": "updates.OrderDate",
    "Day": "updates.Day",
    "Month": "updates.Month",
    "Year": "updates.Year",
    "mmmyyyy": "updates.mmmyyyy",
    "yyyymm": "updates.yyyymm"
}

)\
.execute()

StatementMeta(, f1faf6aa-3ec2-49b0-8dc9-20c88f5228d1, 20, Finished, Available, Finished)

# Defining customer dimension table


In [1]:
from pyspark.sql.types import * 
from delta.tables import * 

# create a customer gold dimension table 
DeltaTable.createIfNotExists(spark)\
.tableName("dimcustomer_gold")\
.addColumn("CustomerName", StringType())\
.addColumn("Email", StringType()) \
.addColumn("First", StringType()) \
.addColumn("Last", StringType()) \
.addColumn("CustomerID", LongType()) \
.execute()

StatementMeta(, 5513f65f-7feb-4b25-aaa2-0cc93b52509b, 3, Finished, Available, Finished)

Extracting First and Last Name from Customer Name 

In [4]:
from pyspark.sql.functions import *

# creating a customer silver dataframe by dropping duplicates and splitting first and last name 

dfdimCustomer_silver = df.dropDuplicates(["CustomerName","Email"]).select(col("CustomerName"), col("Email"))\
.withColumn("First", split(col("CustomerName")," ").getItem(0))\
.withColumn("Last", split(col("CustomerName")," ").getItem(1))

# Display the newy updated customer Silver table
display(dfdimCustomer_silver)

StatementMeta(, 5513f65f-7feb-4b25-aaa2-0cc93b52509b, 6, Finished, Available, Finished)

SynapseWidget(Synapse.DataFrame, a18dddfe-d7f1-4926-a29a-ec72d2ba1095)

# create a customer ID

In [5]:
from pyspark.sql.functions import monotonically_increasing_id, col, when, coalesce, max, lit
    
dfdimCustomer_temp = spark.read.table("dimCustomer_gold")
    
MAXCustomerID = dfdimCustomer_temp.select(coalesce(max(col("CustomerID")),lit(0)).alias("MAXCustomerID")).first()[0]
    
dfdimCustomer_gold = dfdimCustomer_silver.join(dfdimCustomer_temp,(dfdimCustomer_silver.CustomerName == dfdimCustomer_temp.CustomerName) & (dfdimCustomer_silver.Email == dfdimCustomer_temp.Email), "left_anti")
    
dfdimCustomer_gold = dfdimCustomer_gold.withColumn("CustomerID",monotonically_increasing_id() + MAXCustomerID + 1)

# Display the first 10 rows of the dataframe to preview your data

display(dfdimCustomer_gold.head(10))

StatementMeta(, 5513f65f-7feb-4b25-aaa2-0cc93b52509b, 7, Finished, Available, Finished)

SynapseWidget(Synapse.DataFrame, 34894072-d813-40f9-811d-99790582af63)

In [6]:
from delta.tables import *

deltaTable = DeltaTable.forPath(spark, 'Tables/dbo/dimcustomer_gold')
    
dfUpdates = dfdimCustomer_gold
    
deltaTable.alias('gold') \
  .merge(
    dfUpdates.alias('updates'),
    'gold.CustomerName = updates.CustomerName AND gold.Email = updates.Email'
  ) \
   .whenMatchedUpdate(set =
    {
          
    }
  ) \
 .whenNotMatchedInsert(values =
    {
      "CustomerName": "updates.CustomerName",
      "Email": "updates.Email",
      "First": "updates.First",
      "Last": "updates.Last",
      "CustomerID": "updates.CustomerID"
    }
  ) \
  .execute()

StatementMeta(, 5513f65f-7feb-4b25-aaa2-0cc93b52509b, 8, Finished, Available, Finished)

# create a product dimnesion table

In [8]:
from pyspark.sql.types import *
from delta.tables import *

deltaTable.createIfNotExists(spark) \
.tableName("dimproduct_gold") \
.addColumn("ItemName", StringType())\
.addColumn("ItemID", LongType()) \
.addColumn("ItemInfo", StringType())\
.execute()

StatementMeta(, 5513f65f-7feb-4b25-aaa2-0cc93b52509b, 10, Finished, Available, Finished)

In [9]:
from pyspark.sql.functions import col, split, lit, when
    
# Create product_silver dataframe
    
dfdimProduct_silver = df.dropDuplicates(["Item"]).select(col("Item")) \
    .withColumn("ItemName",split(col("Item"), ", ").getItem(0)) \
    .withColumn("ItemInfo",when((split(col("Item"), ", ").getItem(1).isNull() | (split(col("Item"), ", ").getItem(1)=="")),lit("")).otherwise(split(col("Item"), ", ").getItem(1))) 
    
# Display the first 10 rows of the dataframe to preview your data

display(dfdimProduct_silver.head(10))

StatementMeta(, 5513f65f-7feb-4b25-aaa2-0cc93b52509b, 11, Finished, Available, Finished)

SynapseWidget(Synapse.DataFrame, 9279dbbf-9d91-4088-be12-c0181635758b)

In [11]:
from pyspark.sql.functions import monotonically_increasing_id, col, lit, max, coalesce
    
#dfdimProduct_temp = dfdimProduct_silver
dfdimProduct_temp = spark.read.table("dimProduct_gold")
    
MAXProductID = dfdimProduct_temp.select(coalesce(max(col("ItemID")),lit(0)).alias("MAXItemID")).first()[0]
    
dfdimProduct_gold = dfdimProduct_silver.join(dfdimProduct_temp,(dfdimProduct_silver.ItemName == dfdimProduct_temp.ItemName) & (dfdimProduct_silver.ItemInfo == dfdimProduct_temp.ItemInfo), "left_anti")
    
dfdimProduct_gold = dfdimProduct_gold.withColumn("ItemID",monotonically_increasing_id() + MAXProductID + 1)
    
# Display the first 10 rows of the dataframe to preview your data

display(dfdimProduct_gold.head(10))

StatementMeta(, 5513f65f-7feb-4b25-aaa2-0cc93b52509b, 13, Finished, Available, Finished)

SynapseWidget(Synapse.DataFrame, a9b20cc4-d045-4163-affd-7ffcfa7c68e9)

In [13]:
from delta.tables import *
    
deltaTable = DeltaTable.forPath(spark, 'Tables/dbo/dimproduct_gold')
            
dfUpdates = dfdimProduct_gold
            
deltaTable.alias('gold') \
  .merge(
        dfUpdates.alias('updates'),
        'gold.ItemName = updates.ItemName AND gold.ItemInfo = updates.ItemInfo'
        ) \
        .whenMatchedUpdate(set =
        {
               
        }
        ) \
        .whenNotMatchedInsert(values =
         {
          "ItemName": "updates.ItemName",
          "ItemInfo": "updates.ItemInfo",
          "ItemID": "updates.ItemID"
          }
          ) \
          .execute()

StatementMeta(, 5513f65f-7feb-4b25-aaa2-0cc93b52509b, 15, Finished, Available, Finished)

# creating the fact table - sales

In [15]:
from pyspark.sql.types import *
from delta.tables import *
    
DeltaTable.createIfNotExists(spark) \
    .tableName("factsales_gold") \
    .addColumn("CustomerID", LongType()) \
    .addColumn("ItemID", LongType()) \
    .addColumn("OrderDate", DateType()) \
    .addColumn("Quantity", IntegerType()) \
    .addColumn("UnitPrice", FloatType()) \
    .addColumn("Tax", FloatType()) \
    .execute()

StatementMeta(, 5513f65f-7feb-4b25-aaa2-0cc93b52509b, 17, Finished, Available, Finished)

In [16]:
from pyspark.sql.functions import col
    
dfdimCustomer_temp = spark.read.table("dimCustomer_gold")
dfdimProduct_temp = spark.read.table("dimProduct_gold")
    
df = df.withColumn("ItemName",split(col("Item"), ", ").getItem(0)) \
    .withColumn("ItemInfo",when((split(col("Item"), ", ").getItem(1).isNull() | (split(col("Item"), ", ").getItem(1)=="")),lit("")).otherwise(split(col("Item"), ", ").getItem(1))) \
    
    
# Create Sales_gold dataframe
    
dffactSales_gold = df.alias("df1").join(dfdimCustomer_temp.alias("df2"),(df.CustomerName == dfdimCustomer_temp.CustomerName) & (df.Email == dfdimCustomer_temp.Email), "left") \
        .join(dfdimProduct_temp.alias("df3"),(df.ItemName == dfdimProduct_temp.ItemName) & (df.ItemInfo == dfdimProduct_temp.ItemInfo), "left") \
    .select(col("df2.CustomerID") \
        , col("df3.ItemID") \
        , col("df1.OrderDate") \
        , col("df1.Quantity") \
        , col("df1.UnitPrice") \
        , col("df1.Tax") \
    ).orderBy(col("df1.OrderDate"), col("df2.CustomerID"), col("df3.ItemID"))
    
# Display the first 10 rows of the dataframe to preview your data
    
display(dffactSales_gold.head(10))

StatementMeta(, 5513f65f-7feb-4b25-aaa2-0cc93b52509b, 18, Finished, Available, Finished)

SynapseWidget(Synapse.DataFrame, 332a6e00-858e-4a82-a8a1-30082fd5c2b2)

In [17]:
from delta.tables import *
    
deltaTable = DeltaTable.forPath(spark, 'Tables/dbo/factsales_gold')
    
dfUpdates = dffactSales_gold
    
deltaTable.alias('gold') \
  .merge(
    dfUpdates.alias('updates'),
    'gold.OrderDate = updates.OrderDate AND gold.CustomerID = updates.CustomerID AND gold.ItemID = updates.ItemID'
  ) \
   .whenMatchedUpdate(set =
    {
          
    }
  ) \
 .whenNotMatchedInsert(values =
    {
      "CustomerID": "updates.CustomerID",
      "ItemID": "updates.ItemID",
      "OrderDate": "updates.OrderDate",
      "Quantity": "updates.Quantity",
      "UnitPrice": "updates.UnitPrice",
      "Tax": "updates.Tax"
    }
  ) \
  .execute()

StatementMeta(, 5513f65f-7feb-4b25-aaa2-0cc93b52509b, 19, Finished, Available, Finished)